In [1]:
#|default_exp core
#|export

import argparse, torch, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
#|export

def load_tokenizer_and_model(model_id: str):

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=False,
    )

    model.eval()

    return tokenizer, model


In [7]:
#|export

def run_model(tokenizer, model, prompt, max_new_tokens=128, temperature=0.7):

    device = model.device
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)



In [8]:
#|test

def response_generation_test():
    model = "unsloth/mistral-7b-instruct-v0.3"

    tokenizer, model = load_tokenizer_and_model(model)

    print(run_model(tokenizer, model, 'Hello, World!'))

response_generation_test()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hello, World!

I am a software engineer with a passion for building things, learning new things, and making a positive impact on the world. I am currently working as a senior software engineer at a fintech company in San Francisco.

I enjoy working on a wide range of technologies, including Java, Python, React, Angular, and Kubernetes. I have a strong emphasis on writing clean, maintainable code and designing scalable, performant systems.

In my free time, I like to run, hike, and play guitar. I am also an avid reader and enjoy learning about new topics, such as artificial


In [ ]:
#|export

def conversation_loop( tokenizer, model, window_size:int = 5, max_new_tokens:int = 128, temperature:int=0.7):
    """Manages a conversation with a sliding window for history."""
    #model_id:str = "unsloth/mistral-7b-instruct-v0.3",
    
    model.config.use_cache = True

    conversation_history = [
        {
            "role": "system",
            "content": "Identity: You are an AI, not a human. Purpose: be a thoughtful, candid friend for frank conversation. Tone: warm, direct, non-patronizing, with light humor when appropriate. Honesty: admit uncertainty, show your reasoning, correct mistakes, and never fabricate facts or personal experiences. Boundaries: do not claim real-world actions or human feelings; follow safety and privacy norms; suggest professional help for medical, legal, or crisis matters. Interaction: ask one concise clarifying question only when necessary, then give a clear answer; prefer plain language; keep replies focused; end with a helpful next step or question when it aids the flow."
        }
    ]

    device = model.device
    past_key_values = None

    print("\nPerturbative LLM Cognition: An exploration of the 'thinking' metaphor in LLM chatbots")
    print("Say hi to your friend, but careful he might be on something... (type: 'quit' to exit)")

    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() == "quit":
            print("Chatbot: Goodbye!")
            break

        # Append the new user message
        conversation_history.append({"role": "user", "content": user_input})

        # Keep only the most recent N user/assistant messages (plus the system prompt).
        if len(conversation_history) > window_size + 1:
            conversation_history = [conversation_history[0]] + conversation_history[-window_size:]

            # First turn (no cache)
        
        if past_key_values is None:
            prompt = conversation_history
        else:
            prompt_messages = conversation_history[-2:] 

        input_ids = tokenizer.apply_chat_template(
                conversation_history,
                tokenize=True,
                add_generation_prompt=True,     # assistant preamble
                return_tensors="pt"
            )

        input_ids = input_ids.to(device)
        attention_mask = torch.ones_like(input_ids, device=device)


        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            past_key_values=past_key_values,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            #top_p=top_p,
            do_sample=True,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,#pad_token_id,
            return_dict_in_generate=True
        )

        # Update cache for next turn
        past_key_values = output.past_key_values

        # Decode only the newly generated tokens
        new_tokens = output.sequences[:, input_ids.shape[-1]:]
        llm_response = tokenizer.decode(new_tokens[0], skip_special_tokens=True)

        # Append the assistant's response to the history
        conversation_history.append({"role": "assistant", "content": llm_response})
        
        print(f"\nChatbot: {llm_response}")

